In [ ]:
# Importação das bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score, roc_curve,
    mean_squared_error, mean_absolute_error, r2_score
)

# Modelos de Classificação
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Modelos de Regressão
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# Balanceamento
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

# Configurações
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline
pd.set_option('display.max_columns', None)

# Seed para reprodutibilidade
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

## 2.1 Carregamento e Preparação dos Dados

In [ ]:
# Carregar dados
print("Carregando dados...")
flights = pd.read_csv('flights.csv')
airlines = pd.read_csv('airlines.csv')
airports = pd.read_csv('airports.csv')

print(f"Shape dos dados: {flights.shape}")
print(f"\nColunas disponíveis:")
print(flights.columns.tolist())

In [ ]:
# Amostragem para facilitar processamento (opcional)
# Se o dataset for muito grande, podemos usar uma amostra
SAMPLE_SIZE = None  # None para usar todos os dados, ou um número como 100000

if SAMPLE_SIZE and len(flights) > SAMPLE_SIZE:
    print(f"Usando amostra de {SAMPLE_SIZE:,} registros")
    flights_sample = flights.sample(n=SAMPLE_SIZE, random_state=RANDOM_STATE)
else:
    print("Usando dataset completo")
    flights_sample = flights.copy()

print(f"Shape final: {flights_sample.shape}")

## 2.2 Feature Engineering

In [ ]:
# Criar novas features
print("Criando novas features...")

df = flights_sample.copy()

# 1. Variável alvo para classificação: atraso sim/não
if 'ARRIVAL_DELAY' in df.columns:
    df['IS_DELAYED'] = (df['ARRIVAL_DELAY'] > 15).astype(int)  # >15 min = atraso
    print(f"✓ Variável IS_DELAYED criada")
    print(f"  Voos atrasados: {df['IS_DELAYED'].sum():,} ({(df['IS_DELAYED'].mean()*100):.2f}%)")

# 2. Período do dia
if 'SCHEDULED_DEPARTURE' in df.columns:
    df['DEPARTURE_HOUR'] = df['SCHEDULED_DEPARTURE'] // 100
    df['DEPARTURE_PERIOD'] = pd.cut(df['DEPARTURE_HOUR'], 
                                    bins=[0, 6, 12, 18, 24],
                                    labels=['Madrugada', 'Manhã', 'Tarde', 'Noite'])
    print(f"✓ Período do dia criado")

# 3. Final de semana
if 'DAY_OF_WEEK' in df.columns:
    df['IS_WEEKEND'] = df['DAY_OF_WEEK'].isin([6, 7]).astype(int)
    print(f"✓ Indicador de final de semana criado")

# 4. Trimestre/Estação do ano
if 'MONTH' in df.columns:
    df['QUARTER'] = pd.cut(df['MONTH'], 
                           bins=[0, 3, 6, 9, 12],
                           labels=['Q1', 'Q2', 'Q3', 'Q4'])
    
    # Estação do ano (Hemisfério Norte)
    season_map = {1: 'Inverno', 2: 'Inverno', 3: 'Primavera', 4: 'Primavera',
                  5: 'Primavera', 6: 'Verão', 7: 'Verão', 8: 'Verão',
                  9: 'Outono', 10: 'Outono', 11: 'Outono', 12: 'Inverno'}
    df['SEASON'] = df['MONTH'].map(season_map)
    print(f"✓ Trimestre e estação do ano criados")

# 5. Categoria de distância
if 'DISTANCE' in df.columns:
    df['DISTANCE_CATEGORY'] = pd.cut(df['DISTANCE'],
                                     bins=[0, 500, 1000, 2000, 5000],
                                     labels=['Curta', 'Média', 'Longa', 'Muito_Longa'])
    print(f"✓ Categoria de distância criada")

# 6. Tempo de voo programado
if 'SCHEDULED_ARRIVAL' in df.columns and 'SCHEDULED_DEPARTURE' in df.columns:
    # Converter para minutos
    dep_minutes = (df['SCHEDULED_DEPARTURE'] // 100) * 60 + (df['SCHEDULED_DEPARTURE'] % 100)
    arr_minutes = (df['SCHEDULED_ARRIVAL'] // 100) * 60 + (df['SCHEDULED_ARRIVAL'] % 100)
    df['SCHEDULED_FLIGHT_TIME'] = arr_minutes - dep_minutes
    # Ajustar para voos que cruzam a meia-noite
    df.loc[df['SCHEDULED_FLIGHT_TIME'] < 0, 'SCHEDULED_FLIGHT_TIME'] += 1440
    print(f"✓ Tempo de voo programado calculado")

print(f"\nShape após feature engineering: {df.shape}")

In [ ]:
# Visualizar distribuição da variável alvo
if 'IS_DELAYED' in df.columns:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Contagem
    df['IS_DELAYED'].value_counts().plot(kind='bar', ax=axes[0])
    axes[0].set_title('Distribuição de Voos: Atrasados vs Pontuais', fontweight='bold')
    axes[0].set_xlabel('0 = Pontual, 1 = Atrasado')
    axes[0].set_ylabel('Contagem')
    axes[0].set_xticklabels(['Pontual', 'Atrasado'], rotation=0)
    
    # Percentual
    df['IS_DELAYED'].value_counts(normalize=True).plot(kind='pie', ax=axes[1], autopct='%1.1f%%')
    axes[1].set_title('Proporção de Atrasos', fontweight='bold')
    axes[1].set_ylabel('')
    axes[1].legend(['Pontual', 'Atrasado'])
    
    plt.tight_layout()
    plt.show()

## 2.3 Preparação dos Dados para Modelagem

In [ ]:
# Selecionar features para o modelo
print("Selecionando features...")

# Features numéricas
numeric_features = ['MONTH', 'DAY', 'DAY_OF_WEEK', 'DEPARTURE_HOUR', 'DISTANCE', 
                   'SCHEDULED_FLIGHT_TIME', 'IS_WEEKEND']

# Features categóricas
categorical_features = ['AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
                       'DEPARTURE_PERIOD', 'QUARTER', 'SEASON', 'DISTANCE_CATEGORY']

# Verificar quais features existem
numeric_features = [f for f in numeric_features if f in df.columns]
categorical_features = [f for f in categorical_features if f in df.columns]

print(f"Features numéricas: {numeric_features}")
print(f"Features categóricas: {categorical_features}")

# Criar dataset para modelagem
all_features = numeric_features + categorical_features
df_model = df[all_features + ['IS_DELAYED', 'ARRIVAL_DELAY']].copy()

# Remover linhas com NaN na variável alvo
df_model = df_model.dropna(subset=['IS_DELAYED', 'ARRIVAL_DELAY'])

print(f"\nShape para modelagem: {df_model.shape}")
print(f"Missing values por coluna:")
print(df_model.isnull().sum()[df_model.isnull().sum() > 0])

In [ ]:
# Encoding de variáveis categóricas
print("Aplicando encoding...")

df_encoded = df_model.copy()

# Label Encoding para variáveis categóricas
label_encoders = {}
for col in categorical_features:
    if col in df_encoded.columns:
        le = LabelEncoder()
        # Preencher NaN com uma categoria especial
        df_encoded[col] = df_encoded[col].fillna('MISSING')
        df_encoded[col] = le.fit_transform(df_encoded[col].astype(str))
        label_encoders[col] = le
        print(f"✓ {col}: {len(le.classes_)} categorias")

# Preencher NaN em features numéricas com mediana
for col in numeric_features:
    if col in df_encoded.columns:
        if df_encoded[col].isnull().sum() > 0:
            median_val = df_encoded[col].median()
            df_encoded[col].fillna(median_val, inplace=True)
            print(f"✓ {col}: preenchido com mediana = {median_val}")

print(f"\nShape final: {df_encoded.shape}")

## 2.4 Modelagem de Classificação: Prever se Voo vai Atrasar

In [ ]:
# Preparar dados para classificação
print("=" * 80)
print("MODELAGEM DE CLASSIFICAÇÃO")
print("=" * 80)

# Separar features e target
X = df_encoded[numeric_features + categorical_features]
y = df_encoded['IS_DELAYED']

print(f"\nShape de X: {X.shape}")
print(f"Shape de y: {y.shape}")
print(f"\nDistribuição da variável alvo:")
print(y.value_counts())
print(f"\nProporção de classes:")
print(y.value_counts(normalize=True))

In [ ]:
# Split treino/teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

print(f"Tamanho do conjunto de treino: {X_train.shape}")
print(f"Tamanho do conjunto de teste: {X_test.shape}")

# Normalização
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\n✓ Dados normalizados")

In [ ]:
# Definir modelos de classificação
classifiers = {
    'Logistic Regression': LogisticRegression(random_state=RANDOM_STATE, max_iter=1000),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE, n_jobs=-1),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=RANDOM_STATE),
    'XGBoost': XGBClassifier(n_estimators=100, random_state=RANDOM_STATE, eval_metric='logloss'),
    'LightGBM': LGBMClassifier(n_estimators=100, random_state=RANDOM_STATE, verbose=-1)
}

print(f"Modelos a serem treinados: {list(classifiers.keys())}")

In [ ]:
# Treinar e avaliar modelos
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

results_classification = {}

for name, clf in classifiers.items():
    print(f"\n{'='*80}")
    print(f"Treinando: {name}")
    print(f"{'='*80}")
    
    # Treinar
    if name == 'Logistic Regression':
        clf.fit(X_train_scaled, y_train)
        y_pred = clf.predict(X_test_scaled)
        y_pred_proba = clf.predict_proba(X_test_scaled)[:, 1]
    else:
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        y_pred_proba = clf.predict_proba(X_test)[:, 1]
    
    # Métricas
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    
    results_classification[name] = {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'ROC-AUC': roc_auc,
        'model': clf,
        'predictions': y_pred,
        'predictions_proba': y_pred_proba
    }
    
    print(f"Accuracy:  {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1-Score:  {f1:.4f}")
    print(f"ROC-AUC:   {roc_auc:.4f}")
    
    # Matriz de confusão
    print(f"\nMatriz de Confusão:")
    print(confusion_matrix(y_test, y_pred))

print("\n✓ Todos os modelos treinados!")

In [ ]:
# Comparação de modelos
comparison_df = pd.DataFrame(results_classification).T
comparison_df = comparison_df[['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC']]

print("\n" + "="*80)
print("COMPARAÇÃO DE MODELOS - CLASSIFICAÇÃO")
print("="*80)
print(comparison_df.round(4))

# Visualização
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Gráfico de barras
comparison_df.plot(kind='bar', ax=axes[0])
axes[0].set_title('Comparação de Métricas - Modelos de Classificação', fontweight='bold', fontsize=14)
axes[0].set_ylabel('Score')
axes[0].set_xlabel('Modelo')
axes[0].legend(loc='lower right')
axes[0].set_xticklabels(comparison_df.index, rotation=45, ha='right')
axes[0].grid(True, alpha=0.3)

# Heatmap
sns.heatmap(comparison_df, annot=True, fmt='.3f', cmap='YlGnBu', ax=axes[1])
axes[1].set_title('Heatmap de Métricas', fontweight='bold', fontsize=14)

plt.tight_layout()
plt.show()

In [ ]:
# ROC Curves
plt.figure(figsize=(10, 8))

for name in results_classification.keys():
    fpr, tpr, _ = roc_curve(y_test, results_classification[name]['predictions_proba'])
    auc = results_classification[name]['ROC-AUC']
    plt.plot(fpr, tpr, label=f'{name} (AUC = {auc:.3f})', linewidth=2)

plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier')
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curves - Comparação de Modelos', fontsize=14, fontweight='bold')
plt.legend(loc='lower right')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Feature Importance (para modelos baseados em árvore)
best_model_name = comparison_df['F1-Score'].idxmax()
best_model = results_classification[best_model_name]['model']

print(f"\nMelhor modelo (por F1-Score): {best_model_name}")

if hasattr(best_model, 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'Feature': numeric_features + categorical_features,
        'Importance': best_model.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    print("\nTop 15 Features Mais Importantes:")
    print(feature_importance.head(15))
    
    # Visualização
    plt.figure(figsize=(10, 8))
    plt.barh(range(15), feature_importance['Importance'].head(15))
    plt.yticks(range(15), feature_importance['Feature'].head(15))
    plt.xlabel('Importância')
    plt.title(f'Top 15 Features - {best_model_name}', fontweight='bold', fontsize=14)
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

## 2.5 Modelagem de Regressão: Prever Tempo de Atraso

In [ ]:
# Preparar dados para regressão
print("=" * 80)
print("MODELAGEM DE REGRESSÃO")
print("=" * 80)

# Filtrar apenas voos atrasados para regressão
df_delayed = df_encoded[df_encoded['IS_DELAYED'] == 1].copy()

X_reg = df_delayed[numeric_features + categorical_features]
y_reg = df_delayed['ARRIVAL_DELAY']

print(f"\nShape de X (regressão): {X_reg.shape}")
print(f"Shape de y (regressão): {y_reg.shape}")
print(f"\nEstatísticas do atraso:")
print(y_reg.describe())

In [ ]:
# Split treino/teste para regressão
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=RANDOM_STATE
)

print(f"Tamanho do conjunto de treino: {X_train_reg.shape}")
print(f"Tamanho do conjunto de teste: {X_test_reg.shape}")

# Normalização
scaler_reg = StandardScaler()
X_train_reg_scaled = scaler_reg.fit_transform(X_train_reg)
X_test_reg_scaled = scaler_reg.transform(X_test_reg)

print("\n✓ Dados normalizados")

In [ ]:
# Definir modelos de regressão
regressors = {
    'Linear Regression': LinearRegression(),
    'Ridge': Ridge(random_state=RANDOM_STATE),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=RANDOM_STATE, n_jobs=-1),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=RANDOM_STATE),
    'XGBoost': XGBRegressor(n_estimators=100, random_state=RANDOM_STATE),
    'LightGBM': LGBMRegressor(n_estimators=100, random_state=RANDOM_STATE, verbose=-1)
}

print(f"Modelos de regressão a serem treinados: {list(regressors.keys())}")

In [ ]:
# Treinar e avaliar modelos de regressão
results_regression = {}

for name, reg in regressors.items():
    print(f"\n{'='*80}")
    print(f"Treinando: {name}")
    print(f"{'='*80}")
    
    # Treinar
    if name in ['Linear Regression', 'Ridge']:
        reg.fit(X_train_reg_scaled, y_train_reg)
        y_pred_reg = reg.predict(X_test_reg_scaled)
    else:
        reg.fit(X_train_reg, y_train_reg)
        y_pred_reg = reg.predict(X_test_reg)
    
    # Métricas
    mae = mean_absolute_error(y_test_reg, y_pred_reg)
    mse = mean_squared_error(y_test_reg, y_pred_reg)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test_reg, y_pred_reg)
    
    results_regression[name] = {
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'R²': r2,
        'model': reg,
        'predictions': y_pred_reg
    }
    
    print(f"MAE:  {mae:.2f} minutos")
    print(f"MSE:  {mse:.2f}")
    print(f"RMSE: {rmse:.2f} minutos")
    print(f"R²:   {r2:.4f}")

print("\n✓ Todos os modelos de regressão treinados!")

In [ ]:
# Comparação de modelos de regressão
comparison_reg_df = pd.DataFrame(results_regression).T
comparison_reg_df = comparison_reg_df[['MAE', 'RMSE', 'R²']]

print("\n" + "="*80)
print("COMPARAÇÃO DE MODELOS - REGRESSÃO")
print("="*80)
print(comparison_reg_df.round(2))

# Visualização
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Gráfico de barras
comparison_reg_df[['MAE', 'RMSE']].plot(kind='bar', ax=axes[0])
axes[0].set_title('Comparação de Erros - Modelos de Regressão', fontweight='bold', fontsize=14)
axes[0].set_ylabel('Erro (minutos)')
axes[0].set_xlabel('Modelo')
axes[0].legend(loc='upper right')
axes[0].set_xticklabels(comparison_reg_df.index, rotation=45, ha='right')
axes[0].grid(True, alpha=0.3)

# R² Score
comparison_reg_df['R²'].plot(kind='bar', ax=axes[1], color='green')
axes[1].set_title('R² Score - Modelos de Regressão', fontweight='bold', fontsize=14)
axes[1].set_ylabel('R² Score')
axes[1].set_xlabel('Modelo')
axes[1].set_xticklabels(comparison_reg_df.index, rotation=45, ha='right')
axes[1].axhline(y=0, color='r', linestyle='--', alpha=0.5)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Gráfico de resíduos para o melhor modelo
best_reg_model_name = comparison_reg_df['R²'].idxmax()
best_reg_predictions = results_regression[best_reg_model_name]['predictions']

print(f"\nMelhor modelo de regressão (por R²): {best_reg_model_name}")

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Valores preditos vs reais
axes[0].scatter(y_test_reg, best_reg_predictions, alpha=0.3)
axes[0].plot([y_test_reg.min(), y_test_reg.max()], 
             [y_test_reg.min(), y_test_reg.max()], 'r--', lw=2)
axes[0].set_xlabel('Atraso Real (minutos)', fontsize=12)
axes[0].set_ylabel('Atraso Predito (minutos)', fontsize=12)
axes[0].set_title(f'Predições vs Valores Reais - {best_reg_model_name}', 
                  fontweight='bold', fontsize=14)
axes[0].grid(True, alpha=0.3)

# Resíduos
residuals = y_test_reg - best_reg_predictions
axes[1].scatter(best_reg_predictions, residuals, alpha=0.3)
axes[1].axhline(y=0, color='r', linestyle='--', lw=2)
axes[1].set_xlabel('Atraso Predito (minutos)', fontsize=12)
axes[1].set_ylabel('Resíduos', fontsize=12)
axes[1].set_title('Gráfico de Resíduos', fontweight='bold', fontsize=14)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2.6 Conclusões da Modelagem Supervisionada

### Classificação (Prever se voo vai atrasar):
- Comparamos 5 algoritmos diferentes
- Métricas avaliadas: Accuracy, Precision, Recall, F1-Score, ROC-AUC
- Identificamos as features mais importantes para predição

### Regressão (Prever tempo de atraso):
- Testamos 6 modelos diferentes
- Métricas avaliadas: MAE, RMSE, R²
- Análise de resíduos para validar o modelo

### Limitações:
1. Possível desbalanceamento de classes na classificação
2. Necessidade de mais features contextuais (clima, feriados, eventos)
3. Possíveis outliers nos dados de atraso

### Próximos Passos:
1. Otimização de hiperparâmetros com GridSearch
2. Tratamento de desbalanceamento com SMOTE/undersampling
3. Feature engineering mais avançado
4. Ensemble de modelos
5. Análise de erros e casos extremos

In [ ]:
# Salvar resultados
import pickle

print("Salvando modelos e resultados...")

# Salvar melhor modelo de classificação
with open('best_classifier.pkl', 'wb') as f:
    pickle.dump(results_classification[best_model_name]['model'], f)
print(f"✓ Melhor classificador salvo: {best_model_name}")

# Salvar melhor modelo de regressão
with open('best_regressor.pkl', 'wb') as f:
    pickle.dump(results_regression[best_reg_model_name]['model'], f)
print(f"✓ Melhor regressor salvo: {best_reg_model_name}")

# Salvar scalers
with open('scaler_classification.pkl', 'wb') as f:
    pickle.dump(scaler, f)
with open('scaler_regression.pkl', 'wb') as f:
    pickle.dump(scaler_reg, f)
print("✓ Scalers salvos")

# Salvar resultados em CSV
comparison_df.to_csv('classification_results.csv')
comparison_reg_df.to_csv('regression_results.csv')
print("✓ Resultados salvos em CSV")

print("\n" + "="*80)
print("MODELAGEM SUPERVISIONADA CONCLUÍDA!")
print("="*80)